#Donwload a ImageCollection 

## 1._ Install libraries

In [0]:
!pip install earthengine-api

## 2._ Establish connection

In [0]:
!earthengine authenticate

In [0]:
import ee
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
  print("Unexpected error:", sys.exc_info()[0])
  raise

## 3._ Example

In [0]:
pt = ee.Geometry.Point([-2.40986111110000012, 26.76033333330000019])
region = pt.buffer(10)

col = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
        .filterDate('2015-01-01','2015-04-30')\
        .filterBounds(region)
        

## 4._ Define bands 

In [0]:
bands = ['B4','B5'] #Change it!


##5.  Function to pass to Iterate

In [0]:
def accumulate(image,img):
  name_image = image.get('system:index')
  image = image.select([0],[name_image])
  cumm = ee.Image(img).addBands(image)
  return cumm


In [0]:
for band in bands:
  col_band = col.map(lambda img: img.select(band)\
                               .set('system:time_start', img.get('system:time_start'))\
                               .set('system:index', img.get('system:index')))
  #  ImageCollection to List           
  col_list = col_band.toList(col_band.size())

  #  Define the initial value for iterate.
  base = ee.Image(col_list.get(0))
  base_name = base.get('system:index')
  base = base.select([0], [base_name])
  
  #  Eliminate the image 'base'.
  new_col = ee.ImageCollection(col_list.splice(0,1))

  img_cummulative = ee.Image(new_col.iterate(accumulate,base))

  task = ee.batch.Export.image.toDrive(
      image = img_cummulative.clip(region),
      folder = 'landsat',
      fileNamePrefix = band,
      scale = 30).start()  
  
  print('Export Image '+ band+ ' was submitted, please wait ...')

Export Image B4 was submitted, please wait ...
Export Image B5 was submitted, please wait ...


In [0]:
img_cummulative.bandNames().getInfo()

['LC08_197041_20150422',
 'LC08_197042_20150422',
 'LC08_198041_20150107',
 'LC08_198041_20150208',
 'LC08_198041_20150224',
 'LC08_198041_20150312',
 'LC08_198041_20150328',
 'LC08_198041_20150413',
 'LC08_198041_20150429']